In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!export CUDA_VISIBLE_DEVICES=1

In [ ]:
from generate import *
import mmcv
from avcv.visualize import *
from avcv.utils import *
from projector import project

In [ ]:
# def expand_seed(seeds, vector_size):
#     result = []

#     for seed in seeds:
#     rnd = np.random.RandomState(seed)
#     result.append( rnd.randn(1, vector_size) ) 
#     return result


In [ ]:
network_pkl = 'training-runs/00001-dest_gtsr-auto8/network-snapshot-018144.pkl'

device = torch.device('cuda:1')
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

In [ ]:
seeds = [0,1,2,3]
outdir = '.cache/00/'
truncation_psi = 1
# !mkdir -p {outdir}
mmcv.mkdir_or_exist(outdir)
noise_mode = 'const'

In [ ]:
label = torch.zeros([1, G.c_dim], device=device)

In [ ]:
# Generate images
def gen_z(seed, trunc=None):
    
    z = np.random.RandomState(seed).randn(1, G.z_dim)
    if trunc is not None:
        z = np.clip(z, -trunc, trunc)
    
    return torch.from_numpy(z).to(device)
def gen_img(z):
    img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
    return img

for seed_idx, seed in enumerate(seeds):
    print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
    z = gen_z(seed)
    img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.jpg')

In [ ]:
# rand_imgs = []
# labels = []
# rand_zs = []
# for i in range(1, 512, 5):
#     z = gen_z(i)
#     rand_zs.append(z)
#     rand_imgs += [gen_img(z)]
#     labels += [i]
# plot_images(rand_imgs, labels, space=[1, 1])

In [ ]:
from tqdm import tqdm
def gen_in_between(z1, z2,num_in_between=100):
    imgs = []

    delta = (z2-z1)/num_in_between
    for i in tqdm(range(num_in_between)):
        z = z1+i*delta
        img = gen_img(z)
        imgs.append(img[...,::-1])
    return imgs

def gen_with_direction(z1, delta, num):
    imgs = []
    for i in tqdm(range(-num//2, num//2)):
        z = z1+i*delta
        img = gen_img(z)
        imgs.append(img[...,::-1])
    return imgs


# Generate one class

In [ ]:
from glob import glob
from PIL import Image
paths = glob('/data/tsd/classification/GTSRB/train/0/*')
imgs = [mmcv.imread(path, channel_order='rgb') for path in paths]
imgs = [mmcv.imresize(img, (64,64)) for img in imgs]
imgs = [torch.from_numpy(img).float().permute(2,0,1).to(device) for img in imgs]
Image.open(paths[0])

In [ ]:
z0 = gen_z(0)

In [ ]:
min_val = z0.min()
max_val = z0.max()
diff = max_val - min_val

In [ ]:
def put_text(image, pos, text, color=(255, 255, 255), scale=2):
    import cv2
    return cv2.putText(image, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 0.3,
                       color, scale)

In [ ]:
# from tqdm import tqdm
# laten_idx = 0
# imgs = []
# step = 60
# for laten_idx in tqdm(range(0, 512)):
#     for i in range(step):
#         _z = z0

#         _z[0, laten_idx] = min_val + i*diff

#         img = gen_img(_z)
#         img = put_text(img.copy(), (2, 15),f'{laten_idx:03d}', scale=1, )
#         imgs += [img[...,::-1]]
# # show(img, dpi=10)
# images_to_video(imgs, '.cache/t2.mp4')

In [ ]:
imgs = []
lbls = []
for i in range(100):
    z = gen_z(i)
    img = gen_img(z)
    imgs += [img]
    lbls.append(i)
    if len(imgs) == 25:
        plot_images(imgs[:25], lbls[:25], dpi=150)
        imgs, lbls = [], []

In [ ]:
z1 = gen_z(87, trunc=None)
z2 = gen_z(86, trunc=None)
imgs = gen_in_between(z1, z2, 100)
images_to_video(imgs, '.cache/t2.mp4')

In [ ]:
# distance = (z1-z2)